# 3-D

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import jax
import jax.numpy as jnp
import seaborn as sns

from dosipy.bhte import init_temp, delta_temp_analytic, temp_3d
from dosipy.constants import eps_0, mu_0
from dosipy.field import efield, hfield
from dosipy.utils.dataloader import (load_tissue_diel_properties,
                                     load_antenna_el_properties)
from dosipy.utils.integrate import elementwise_dblquad
from dosipy.utils.viz import fig_config, set_colorblind

In [ ]:
print(f'platform: {jax.lib.xla_bridge.get_backend().platform}')

In [ ]:
# jax.config.update("jax_enable_x64", True)

In [ ]:
set_colorblind()
%config InlineBackend.figure_format = 'retina'

In [ ]:
# dry skin bhte parameters from IT'IS database
# https://itis.swiss/virtual-population/tissue-properties/database/
k = 0.37       # thermal conductivity [W/m/°C]
rho = 1109.    # dry skin density [kg/m^3]
C = 3391.      # specific heat of dry skin [Ws/kg/°C]
m_b = 1.76e-6  # blood perfusion [m^3/kg/s], equivalent to 106 mL/min/kg

# for blood
k_b = 0.52     # thermal conductivity [W/m/°C]
rho_b = 1000.  # kg/m^3
C_b = 3617.    # specific heat of blood [J/kg/°C]

# additional parameters
h_0 = 10.      # heat convection coefficient skin surface - air [W/m^2/°C]
T_a = 37.      # arterial temperature [°C]
T_c = 37.      # body core temperature [°C]
T_f = 25.      # surrounding air temperature [°C]
Q_m = 6385.    # metabolic heat generation [W/m^3]

## 3-D bioheat transfer equation in time

Comparisson of the spectral method time domain solution and the analytic solution proposed in Foster et al. 2016.
This time, the model for the PSTD method is three-dimensional. Additionally, realistic antenna is considered -- half-wave dipole centrally powered by a voltage source set to 1V.

### Setup

In [ ]:
# frequency
f = 10e9

# conductivity, relative permitivity, tangent loss and penetration depth
sigma, eps_r, tan_loss, pen_depth = load_tissue_diel_properties('skin_dry', f)

# `pen_depth` is the energy penetration depth into tissue, which is defined as
# the distance beneath the surface at which the SAR has fallen to a factor of
# 1/e below that at the surface; one-half of the more commonly reported wave
# penetration depth
pen_depth = pen_depth / 2

# air (vacuum) resistance 
Z_air = np.sqrt(mu_0 / eps_0)

# dry skin resistance
Z_skin_dry = np.sqrt(mu_0 / (eps_r * eps_0))

# energy (Fresnel) transmission coefficient into the tissue
T_tr = 2 * Z_skin_dry / (Z_air + Z_skin_dry)

# antenna electric properties (Poljak 2005)
antenna_data = load_antenna_el_properties(f)
Is = antenna_data.ireal.to_numpy() + antenna_data.iimag.to_numpy() * 1j

# antenna position -- coordinates
xs = antenna_data.x.to_numpy()
xs = jnp.asarray(xs)
ys = jnp.zeros_like(xs)
zs = jnp.zeros_like(xs)

# effective radiated skin area
target_area = (0.02, 0.02)  # 2 x 2 cm2
A = target_area[0] * target_area[1]
h = -5 / 1000  # distance from the antenna
N = [11, 11, 11]
xt = jnp.linspace(-target_area[0]/2, target_area[0]/2, N[0]) + xs[-1] / 2
yt = jnp.linspace(-target_area[1]/2, target_area[1]/2, N[1])
zt = jnp.linspace(h, h - pen_depth, N[2])

### EM-field

Free space electric and magnetic field distribution.

In [ ]:
Ex = np.empty((xt.size, yt.size, zt.size), dtype=np.complex128)
Ey = np.empty((xt.size, yt.size, zt.size), dtype=np.complex128)
Ez = np.empty((xt.size, yt.size, zt.size), dtype=np.complex128)
for x_idx, _xt in enumerate(xt):
    for y_idx, _yt in enumerate(yt):
        for z_idx, _zt in enumerate(zt):
            _Ex, _Ey, _Ez = efield(_xt, _yt, _zt, xs, ys, zs, Is, f)
            Ex[x_idx, y_idx, z_idx] = _Ex
            Ey[x_idx, y_idx, z_idx] = _Ey
            Ez[x_idx, y_idx, z_idx] = _Ez
E = np.abs(np.sqrt(Ex ** 2 + Ey ** 2 + Ez ** 2))
SAR = sigma * E ** 2 / (2 * rho)

### Simulation

Solution of the 3-D bio-heat transfer equation in time.

In [ ]:
sim_time = 360
tg = 101
t = np.linspace(0, sim_time, tg)
T = temp_3d(t, N, target_area, pen_depth, k, rho, C, rho_b, C_b, m_b, h_0, T_a,
            T_c, T_f, Q_m, SAR)
deltaT = T[-1, :, :, 0] - T[0, :, :, 0]

### Visualization

Temperature change on the surface.

In [ ]:
extent=(xt.min(), xt.max(), yt.min(), yt.max())
interp = 'none'

fig_config(scaler=2)
fig = plt.figure()
ax = fig.add_subplot()
cs = ax.imshow(deltaT, extent=extent, interpolation=interp, cmap='viridis')
cbar = fig.colorbar(cs, ax=ax)
cbar.ax.set_ylabel('$\\Delta T_{sur}$ [°C]')
ax.plot(xs, ys, 'r-', linewidth=4,
        label=(f'$\\lambda/2$ dipole antenna\ndistance = {-h * 1000} mm'))
ax.set(xlabel='$x$ [m]',
       ylabel='$y$ [m]',
       xticks=[xt.min(), (xt.min() + xt.max()) / 2, xt.max()],
       yticks=[-0.01, -0.005, 0.0, 0.005, 0.01])
ax.legend()
fig.tight_layout();